# Text Classification with 1-d CNN

이전 실습 중에 LSTM을 이용하여 Text를 state vector로 encoding하고, 이를 Feature 로 활용하여 Text classification하는 IMDB dataset 기반 sentimental analysis task를 다루어 본 바 있습니다.  

본 실습에서는 동일한 task를 1-d CNN Encoder를 활용하여 다시 다루어 보겠습니다. 이를 통해 Feature를 추출하는 관점에 따른 모델의 특성 및 분류성능 차이를 확인해 보겠습니다.

(참고)  
https://github.com/gilbutITbook/006975/blob/master/6.4-sequence-processing-with-convnets.ipynb  


In [1]:
# %matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

In [16]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [21]:
# import imdb
from keras.datasets import imdb

# 데이터 관련 설정은 LSTM 케이스와 동일하게 한다.
max_features = 10000
max_tokens = 580
embedding_size = 8

# save np.load
np_load_old = np.load
# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
# call load_data with allow_pickle implicitly set to true
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)   # 원래는 이 라인만 있으면 된다.
# restore np.load for future normal usage
np.load = np_load_old

print("Train-set size: ", len(x_train))
print("Test-set size:  ", len(x_test))

Train-set size:  25000
Test-set size:   25000


In [22]:
pad = 'pre'
x_train_pad = pad_sequences(x_train, maxlen=max_tokens, padding=pad, truncating=pad)
x_test_pad = pad_sequences(x_test, maxlen=max_tokens, padding=pad, truncating=pad)
x_train_pad.shape

(25000, 580)

In [23]:
x_train_pad[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

데이터셋 구성은 LSTM 모델의 경우와 동일합니다.  

## Create Model with Conv1D

In [28]:
model = Sequential()
model.add(Embedding(input_dim=max_features,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))
model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 7, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1))

model.summary()

model.compile(optimizer=Adam(lr=1e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 580, 8)            80000     
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 574, 32)           1824      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 114, 32)           0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 108, 32)           7200      
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 89,057
Trainable params: 89,057
Non-trainable params: 0
_________________________________________________________________


In [29]:
layer_outputs = [layer.output for layer in model.layers]
layer_outputs

[<tf.Tensor 'layer_embedding_1/embedding_lookup/Identity_1:0' shape=(?, 580, 8) dtype=float32>,
 <tf.Tensor 'conv1d_11/Relu:0' shape=(?, 574, 32) dtype=float32>,
 <tf.Tensor 'max_pooling1d_5/Squeeze:0' shape=(?, 114, 32) dtype=float32>,
 <tf.Tensor 'conv1d_12/Relu:0' shape=(?, 108, 32) dtype=float32>,
 <tf.Tensor 'global_max_pooling1d_4/Max:0' shape=(?, 32) dtype=float32>,
 <tf.Tensor 'dense_4/BiasAdd:0' shape=(?, 1) dtype=float32>]

In [30]:
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=100, batch_size=64)

Train on 23750 samples, validate on 1250 samples
Epoch 1/100
23750/23750 [==============================] - 3s 107us/sample - loss: 1.4042 - acc: 0.5000 - val_loss: 1.1238 - val_acc: 0.5008
Epoch 2/100
23750/23750 [==============================] - 2s 92us/sample - loss: 1.0121 - acc: 0.5000 - val_loss: 0.9251 - val_acc: 0.5008
Epoch 3/100
23750/23750 [==============================] - 2s 91us/sample - loss: 0.8702 - acc: 0.5000 - val_loss: 0.8232 - val_acc: 0.5008
Epoch 4/100
23750/23750 [==============================] - 2s 92us/sample - loss: 0.7900 - acc: 0.5000 - val_loss: 0.7611 - val_acc: 0.5008
Epoch 5/100
23750/23750 [==============================] - 2s 95us/sample - loss: 0.7405 - acc: 0.5000 - val_loss: 0.7238 - val_acc: 0.5008
Epoch 6/100
23750/23750 [==============================] - 2s 92us/sample - loss: 0.7120 - acc: 0.5000 - val_loss: 0.7040 - val_acc: 0.5008
Epoch 7/100
23750/23750 [==============================] - 2s 92us/sample - loss: 0.6982 - acc: 0.5008 - val_l

23750/23750 [==============================] - 2s 100us/sample - loss: 0.3852 - acc: 0.8486 - val_loss: 0.4668 - val_acc: 0.8176
Epoch 59/100
23750/23750 [==============================] - 2s 93us/sample - loss: 0.3795 - acc: 0.8514 - val_loss: 0.4626 - val_acc: 0.8192
Epoch 60/100
23750/23750 [==============================] - 2s 92us/sample - loss: 0.3733 - acc: 0.8544 - val_loss: 0.4587 - val_acc: 0.8208
Epoch 61/100
23750/23750 [==============================] - 2s 91us/sample - loss: 0.3667 - acc: 0.8567 - val_loss: 0.4539 - val_acc: 0.8216
Epoch 62/100
23750/23750 [==============================] - 2s 101us/sample - loss: 0.3601 - acc: 0.8585 - val_loss: 0.4510 - val_acc: 0.8224
Epoch 63/100
23750/23750 [==============================] - 2s 93us/sample - loss: 0.3547 - acc: 0.8608 - val_loss: 0.4664 - val_acc: 0.8208
Epoch 64/100
23750/23750 [==============================] - 2s 94us/sample - loss: 0.3480 - acc: 0.8635 - val_loss: 0.4718 - val_acc: 0.8256
Epoch 65/100
23750/23750

In [31]:
result = model.evaluate(x_test_pad, y_test)

25000/25000 [==============================] - 2s 69us/sample - loss: 0.5082 - acc: 0.8506


In [32]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 85.06%
